<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Manipulate-Hamiltonian-into-blocks" data-toc-modified-id="Manipulate-Hamiltonian-into-blocks-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Manipulate Hamiltonian into blocks</a></span></li><li><span><a href="#Feature-computation" data-toc-modified-id="Feature-computation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Feature computation</a></span></li><li><span><a href="#Feature-Preprocessing" data-toc-modified-id="Feature-Preprocessing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Feature Preprocessing</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Dataset</a></span></li><li><span><a href="#DataLoader" data-toc-modified-id="DataLoader-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>DataLoader</a></span></li><li><span><a href="#Model" data-toc-modified-id="Model-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Model</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Evaluation</a></span></li><li><span><a href="#Tests" data-toc-modified-id="Tests-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Tests</a></span><ul class="toc-item"><li><span><a href="#set-up-wigner-d-rotations-matrices" data-toc-modified-id="set-up-wigner-d-rotations-matrices-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>set up wigner-d rotations matrices</a></span></li><li><span><a href="#block-wise-rotations" data-toc-modified-id="block-wise-rotations-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>block wise rotations</a></span></li><li><span><a href="#Rotate-matrix" data-toc-modified-id="Rotate-matrix-9.3"><span class="toc-item-num">9.3&nbsp;&nbsp;</span>Rotate matrix</a></span></li><li><span><a href="#Eigenvalue-tests" data-toc-modified-id="Eigenvalue-tests-9.4"><span class="toc-item-num">9.4&nbsp;&nbsp;</span>Eigenvalue tests</a></span></li><li><span><a href="#check-decoupling-of-blocks" data-toc-modified-id="check-decoupling-of-blocks-9.5"><span class="toc-item-num">9.5&nbsp;&nbsp;</span>check decoupling of blocks</a></span></li><li><span><a href="#check-feature-rotations" data-toc-modified-id="check-feature-rotations-9.6"><span class="toc-item-num">9.6&nbsp;&nbsp;</span>check feature rotations</a></span></li><li><span><a href="#mean-Wei-Bin's-tests" data-toc-modified-id="mean-Wei-Bin's-tests-9.7"><span class="toc-item-num">9.7&nbsp;&nbsp;</span>mean Wei Bin's tests</a></span><ul class="toc-item"><li><span><a href="#meanless-hamiltonian" data-toc-modified-id="meanless-hamiltonian-9.7.1"><span class="toc-item-num">9.7.1&nbsp;&nbsp;</span>meanless hamiltonian</a></span></li><li><span><a href="#meanless-blocks" data-toc-modified-id="meanless-blocks-9.7.2"><span class="toc-item-num">9.7.2&nbsp;&nbsp;</span>meanless blocks</a></span></li><li><span><a href="#stupid-test" data-toc-modified-id="stupid-test-9.7.3"><span class="toc-item-num">9.7.3&nbsp;&nbsp;</span>stupid test</a></span></li></ul></li></ul></li><li><span><a href="#Jigyasa-needs-to-modify-samples-for-features-to-start-from-1" data-toc-modified-id="Jigyasa-needs-to-modify-samples-for-features-to-start-from-1-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Jigyasa needs to modify samples for features to start from 1</a></span></li><li><span><a href="#Old-and-existing" data-toc-modified-id="Old-and-existing-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Old and existing</a></span></li></ul></div>

In [4]:
# %load_ext autoreload
# %autoreload 2

In [166]:
import numpy as np
import torch
import json
import ase.io
from itertools import product
import matplotlib.pyplot as plt
# from rascal.representations import SphericalExpansion
import copy
from tqdm import tqdm
from ase.units import Hartree

from torch_hamiltonian_utils.torch_cg import ClebschGordanReal
from torch_hamiltonian_utils.torch_hamiltonians import fix_pyscf_l1, lowdin_orthogonalize, dense_to_blocks, blocks_to_dense, couple_blocks, decouple_blocks, hamiltonian_features
from torch_hamiltonian_utils.torch_builder import TensorBuilder

import equistore
from equistore import Labels, TensorBlock, TensorMap
from equistore_utils.librascal import  RascalSphericalExpansion, RascalPairExpansion
from equistore_utils.acdc_mini import acdc_standardize_keys, cg_increment, cg_combine
from equistore_utils.model_hamiltonian import get_feat_keys, get_feat_keys_from_uncoupled 

import importlib
torch.set_default_dtype(torch.float64)

In [419]:
def get_feat_keys_from_uncoupled(block_keys, sigma=None, order_nu=None):
    """Map UNCOUPLED block keys to corresponding feature key. take as extra input the sigma, nu value if required.
    sigma=0 returns all possible sigma values at given 'nu'"""
    blocktype, species1, n1, l1, species2, n2, l2 = block_keys
    feat_blocktype = blocktype
    keys_L=[]
    for L in range(abs(l1-l2), l1+l2+1):
        if sigma is None:
            z = (l1+l2+L)%2
            inv_sigma = 1 - 2*z
        elif abs(sigma)==1:
            inv_sigma = sigma
        else: 
            raise("Please check sigma value, it should be +1 or -1")
        
        if blocktype == 1 and n1 == n2 and l1 == l2:
            feat_blocktype = inv_sigma
                 
        if inv_sigma == -1 and blocktype == 0 and n1 == n2 and l1 == l2:
            continue     
        
        keys_L.append([(order_nu, inv_sigma, L, species1, species2, feat_blocktype)])
    #     feat= (blocktype, L,sigma,species1, species2)
    feat = Labels(["order_nu", "inversion_sigma", "spherical_harmonics_l", "species_center", "species_neighbor", "block_type"], np.asarray(keys_L, dtype=np.int32).reshape(-1,6))
    return feat

In [420]:
# frames = ase.io.read("data/water_rotated/water_rotated_3.xyz",":")
# for f in frames:
#     f.cell = [100,100,100]
#     f.positions += 50
    
# jorbs = json.loads(json.load(open('data/water-hamiltonian/water_orbs.json', "r")))
# orbs = {}
# zdic = {"O" : 8, "H":1}
# for k in jorbs:
#     orbs[zdic[k]] = jorbs[k]
# focks = np.load("data/water_rotated/water_rotated_saph_3.npy", allow_pickle=True)[:len(frames)]
# rotations = np.load("data/water_rotated/rotations_3.npy", allow_pickle = True)
# focks2 = np.load("data/ethanol-hamiltonian/ethanol_saph_orthogonal.npy", allow_pickle = True)[:len(frames)]

In [421]:
def lowdin_orthogonalize(fock, s):
    """
    lowdin orthogonalization of a fock matrix computing the square root of the overlap matrix
    """
    eva, eve = np.linalg.eigh(s)
    sm12 = eve @ np.diag(1.0/np.sqrt(eva)) @ eve.T
    return sm12 @ fock @ sm12

In [422]:
frames1 = ase.io.read("data/water-hamiltonian/water_coords_1000.xyz",":50")
# frames2 = ase.io.read("data/ethanol-hamiltonian/ethanol_4500.xyz", ":2")
frames = frames1 #+ frames2
for f in frames:
    f.cell = [100,100,100]
    f.positions += 50
jorbs = json.loads(json.load(open('data/water-hamiltonian/water_orbs.json', "r")))
#jorbs = json.loads(json.load(open('data/water-hamiltonian/orbs_def2_water.json', "r")))
# jorbs = json.load(open('data/ethanol-hamiltonian/orbs_saph_ethanol.json', "r"))
orbs = {}
zdic = {"O" : 8, "H":1, "C":6}
for k in jorbs:
    orbs[zdic[k]] = jorbs[k]
# 
# focks = np.load("data/water-hamiltonian/water_fock.npy", allow_pickle=True)[:len(frames1)]
# overlap = np.load("data/water-hamiltonian/water_overlap.npy", allow_pickle=True)[:len(frames1)]

# orthogonal = []
# for i in range(len(focks)): 
#     focks[i] = fix_pyscf_l1(focks[i],frames[i], orbs)
#     overlap[i] = fix_pyscf_l1(overlap[i],frames[i], orbs)
#     orthogonal.append(lowdin_orthogonalize(focks[i], overlap[i]))
#focks = np.asarray(orthogonal, dtype=np.float64)
focks1 = np.load("data/water-hamiltonian/water_saph_orthogonal.npy", allow_pickle=True)[:len(frames1)]
# focks2 = np.load("data/ethanol-hamiltonian/ethanol_saph_orthogonal.npy", allow_pickle = True)[:len(frames2)]
focks = focks1

In [314]:
# #generating jagged matrix
# f=[]
# for x in focks:
#     f.append(x)
# for x in focks2:
#     f.append(x)
    
# jagged = np.asanyarray(f)

In [315]:
cg = ClebschGordanReal(8)

## Manipulate Hamiltonian into blocks

In [316]:
blocks = dense_to_blocks(focks, frames, orbs)
fock_bc = couple_blocks(blocks, cg)

In [317]:
fock_bc

TensorMap with 8 blocks
keys: ['block_type' 'a_i' 'n_i' 'l_i' 'a_j' 'n_j' 'l_j' 'L']
            0        8     2     0     8     2     0    0
            0        8     2     0     8     2     1    1
            0        8     2     1     8     2     1    0
            0        8     2     1     8     2     1    2
            2        1     1     0     8     2     0    0
            2        1     1     0     8     2     1    1
            0        1     1     0     1     1     0    0
            1        1     1     0     1     1     0    0

## Feature computation

In [318]:
rascal_hypers = {
    "interaction_cutoff": 4.0,
    "cutoff_smooth_width": 0.5,
    "max_radial": 6,
    "max_angular": 4,
    "gaussian_sigma_constant" : 0.2,
    "gaussian_sigma_type": "Constant",
    "compute_gradients":  False,
}

spex = RascalSphericalExpansion(rascal_hypers)
rhoi = spex.compute(frames)

lmax = rascal_hypers["max_angular"]

In [319]:
pairs = RascalPairExpansion(rascal_hypers)
gij = pairs.compute(frames)

In [320]:
rho1i = acdc_standardize_keys(rhoi)
rho1i.keys_to_properties(['species_neighbor'])
gij =  acdc_standardize_keys(gij)

In [321]:
rho2i = cg_increment(rho1i, rho1i, lcut=lmax, other_keys_match=["species_center"], clebsch_gordan=cg)

In [322]:
#rho3i = cg_increment(rho2i, rho1i, lcut=2, other_keys_match=["species_center"], clebsch_gordan=cg)

In [323]:
rho1ij = cg_increment(rho1i, gij, lcut=lmax, other_keys_match=["species_center"], clebsch_gordan=cg)

In [324]:
#rho2ij = cg_increment(rho2i, gij, lcut=2, other_keys_match=["species_center"], clebsch_gordan=cg)

In [325]:
features = hamiltonian_features(rho2i, rho1ij)

In [326]:
features


TensorMap with 45 blocks
keys: ['order_nu' 'inversion_sigma' 'spherical_harmonics_l' 'species_center' 'species_neighbor' 'block_type']
           2             1                    0                   1                1              0
           2             1                    1                   1                1              0
           2             1                    2                   1                1              0
        ...
           2            -1                    4                   1                1              1
           2            -1                    4                   1                1             -1
           2            -1                    4                   1                8              2

In [327]:
from equistore.io import save
save("feature.npz", features)

## Feature Preprocessing

In [328]:
def normalize_feats(feat, all_blocks=True): 
    all_norm = 0
    normalized_blocks=[]
    for block_idx, block in feat: 
        block_norm = np.linalg.norm(block.values)
#         print(block_idx, block_norm)
        all_norm = block_norm**2 * len(block.samples) 
    
        newblock = TensorBlock(
                        values=block.values/np.sqrt(all_norm ),
                        samples=block.samples,
                        components=block.components,
                        properties= block.properties)                    
        normalized_blocks.append(newblock) 
        
    norm_feat = TensorMap(feat.keys, normalized_blocks)
    raise Exception ("Dont do it!")
    return norm_feat

In [329]:
#norm_feat = normalize_feats(features)

In [330]:
# from equistore.io import save
# save("./norm_feat.npz", norm_feat)

## Dataset

In [331]:
from equistore.io import _labels_from_npz
import equistore.operations as operations

class HamiltonianDataset(torch.utils.data.Dataset):
    #Dataset class
    def __init__(self, feature_path, target, frames, feature_nu = 2):
        #
        self.features = np.load(feature_path, mmap_mode = 'r')
        #self.target = np.load(target_path, mmap_mode = 'r') 
        self.target = target #Uncoupled hamiltonian 
        self.keys_features = equistore.io._labels_from_npz(self.features["keys"])
        self.currentkey = self.target.keys[0]
        self.feature_nu = feature_nu
        self.frames = frames
        
        self.allfeatkey = []
        for t_key in self.target.keys:
            feature_key = self.get_feature_keys(t_key)
            self.allfeatkey.append(feature_key)
        #Remove Duplicates
        nodupes = set()
        for x in self.allfeatkey:
            if len(x) > 1:
                for z in x:
                    nodupes.add(tuple(z))
            else:
                nodupes.add(tuple(x[0]))
        
        nodupes = np.array(list(nodupes), np.int32)
        
        self.allfeatkey = Labels(names = self.allfeatkey[0].dtype.names, values = nodupes)
        

    def __len__(self):
        return len(self.frames)
    
    def __getitem__(self, structure_idx):
        feature_block, feature_key = self.generate_feature_block(self.features, structure_idx)        
        #samples_filter, target_block_samples = self.get_index_from_idx(self.target.block(self.currentkey).samples, structure_idx)

        if self.currentkey is None:
            t_blocks = []
            for _, block in self.target:            
                t_block = operations.slice_block(block, samples = Labels(names = ['structure'], values = (np.array(structure_idx)+1).reshape(-1,1)) )
                t_blocks.append(t_block)
            target_block = TensorMap(self.target.keys, t_blocks)
        else:
            target_block = operations.slice_block(self.target.block(self.currentkey), samples = Labels(names = ['structure'], values = (np.array(structure_idx)+1).reshape(-1,1)) )
        structure = [self.frames[i] for i in structure_idx]
        #Modify feature_block to tensormap
        feature_map = TensorMap(feature_key, feature_block)
        return feature_map, target_block, structure


    def get_feature_keys(self,uncoupled_key):
        return get_feat_keys_from_uncoupled(uncoupled_key, order_nu = self.feature_nu)
    
    def generate_feature_block(self, memmap, structure_idx):
        #Generate the block from npz file
        output = []
        if self.currentkey is None:
            feature_key = self.allfeatkey
                
        else:
            feature_key = self.get_feature_keys(self.currentkey)
            
        for key in feature_key:
            block_index = list(self.keys_features).index(key)
            prefix = f"blocks/{block_index}/values"        
            block_samples = equistore.io._labels_from_npz(memmap[f"{prefix}/samples"])
            block_components = []
            for i in range(1):
                block_components.append(equistore.io._labels_from_npz(memmap[f"{prefix}/components/{i}"]))
            block_properties = equistore.io._labels_from_npz(memmap[f"{prefix}/properties"])
             

            samples_filter, block_samples = self.get_index_from_idx(block_samples, structure_idx)

            block_data = memmap[f"{prefix}/data"][samples_filter]
            block = TensorBlock(block_data, block_samples, block_components, block_properties)
            output.append(block)
        return output, feature_key
    
    def get_n_properties(self, memmap, key):
        block_index = list(self.keys_features).index(key)
        prefix = f"blocks/{block_index}/values"  
        block_properties = equistore.io._labels_from_npz(memmap[f"{prefix}/properties"])
        
        return len(block_properties)
    
    def get_index_from_idx(self, block_samples, structure_idx):
        #Get samples label from IDX
        samples = Labels(names = ['structure'], values = np.array(structure_idx).reshape(-1,1))
        
        all_samples = block_samples[['structure']].tolist()
        set_samples_to_slice = set(samples.tolist())
        samples_filter = np.array(
            [sample in set_samples_to_slice for sample in all_samples]
        )
        new_samples = block_samples[samples_filter]
        
        return samples_filter, new_samples
    
    def collate_output_values(blocks):
        feature_out = []
        target_out = []
        for sample_output in blocks:
            feature_block, target_block, structure = sample_output
            for z in feature_block:
                feature_out.append(torch.tensor(z.values))
            target_out.append(torch.tensor(target_block.values))

        return feature_out, target_out

In [332]:
test_target_path = "./test_fock.npz"
test_feature_path = "./norm_feat.npz"
test = HamiltonianDataset(test_feature_path, blocks, frames)

## DataLoader

In [333]:
def collate_blocks(block_tuple):
    feature_tensor_map, target_block, structure_array = block_tuple[0]
    
    return feature_tensor_map, target_block, structure_array
    

In [334]:
from torch.utils.data import DataLoader, BatchSampler, SubsetRandomSampler


#Sampler = torch.utils.data.SubsetRandomSampler(range(1,len(test)+1), generator=None)
Sampler = torch.utils.data.sampler.RandomSampler(test)
BSampler = torch.utils.data.sampler.BatchSampler(Sampler, batch_size = 50, drop_last = False)

dataloader = DataLoader(test, sampler = BSampler, collate_fn = collate_blocks)

## Model 

In [335]:
def get_block_samples(t_key, feature_map):
    f_key = get_feat_keys_from_uncoupled(t_key, None , 2)
    ss = feature_map.block(f_key[0]).samples.copy()
    ss["structure"] = ss["structure"]+1
    
    return ss

In [336]:
class HamModel(torch.nn.Module):
    #Handles prediction of entire hamiltonian and derived results
    def __init__(self, Hamiltonian_Dataset, device, regularization=None, seed=None, layer_size=None):
        super().__init__()
#         self.features = features 
#         self.target = target
        self.models = torch.nn.ModuleDict()
        self.loss_history={}
        self.device = device
        self.target_keys = Hamiltonian_Dataset.target.keys
        self.block_samples = {}
        self.block_components = {}
        for key in Hamiltonian_Dataset.target.keys:
#             _block_type, _a_i, _n_i, _l_i, _a_j, _n_j, _l_j = key
#             target_keys = Hamiltonian_Dataset.target.keys[Hamiltonian_Dataset.target.blocks_matching(
#                 block_type = _block_type, a_i = _a_i, n_i = _n_i, l_i = _l_i, a_j = _a_j,
#                 n_j = _n_j, l_j = _l_j)]
            
            #self.block_samples[str(key)] = Hamiltonian_Dataset.target.block(key).samples
            self.block_components[str(key)] = Hamiltonian_Dataset.target.block(key).components
        
    
            n_inputs = []
            model_keys = []

            feature_keys = Hamiltonian_Dataset.get_feature_keys(key)
            for f_key in feature_keys: 
                n_features = Hamiltonian_Dataset.get_n_properties(Hamiltonian_Dataset.features, f_key)
                n_inputs.append(n_features)
                model_keys.append(f_key)
                
                
            n_outputs = np.ones_like(n_inputs)
                
            self.models[str(key)] = BlockModel(cg.decouple,n_inputs, n_outputs, device, model_keys, key, seed = seed, hidden_layers = layer_size)
        self.to(device)
            
    def forward(self, x):
        #Ham model uses target keys
        pred_blocks = []
        for t_key in self.target_keys:
            
            pred = self.models[str(t_key)].forward(x) #feature_tensormap must correspond to the correct features, model returns block
            
            #try:
#             print (pred.shape)
#             print ((2 * t_key['l_i'])+1)
#             print ((2 * t_key['l_j']) + 1)
            pred_block = TensorBlock(
                    values=pred.reshape((-1, (2 * t_key['l_i'])+1, (2 * t_key['l_j']) + 1, 1)), #?
                    samples = get_block_samples(t_key, x),
                    components = self.block_components[str(t_key)] ,
                    properties= Labels(["dummy"], np.asarray([[0]], dtype=np.int32))
                )
#             except:
#                 print (t_key)
#                 print (pred)
#                 print (self.block_samples[str(t_key)])
#                 print (self.block_components[str(t_key)])
                
            pred_blocks.append(pred_block)
        pred_hamiltonian = TensorMap(self.target_keys, pred_blocks)
        return(pred_hamiltonian)
    
    #write/fix forward function for train_indiv
    
    def train_individual(self, train_dataloader, regularization_dict, optimizer_type, n_epochs, loss_function, lr):
        #Iterates through the keys of self.model, then for each key we will fit self.model[key] with data[key]
        total = len(self.models)
        for index, t_key in enumerate(self.target_keys):
            print ("Now training on Block {} of {}".format(index, total))
            train_dataloader.dataset.currentkey = t_key
            
            loss_history_key = self.models[str(t_key)].fit(train_dataloader, loss_function, optimizer_type, lr, regularization_dict[str(t_key)], n_epochs)

            self.loss_history[str(t_key)] = loss_history_key
    
    def train_collective(self, train_dataloader, regularization_dict, optimizer_type, n_epochs, loss_function, lr):
        #for every loop through target keys, we predict the corresponding block and assemble the final hamiltonian
        optimizer_dict = {}
        if optimizer_type == "Adam":
            for key in train_dataloader.dataset.target.keys:
                optimizer_dict[str(key)] = torch.optim.Adam(self.models[str(key)].parameters(), lr = lr, weight_decay = regularization_dict[str(key)])
            threshold = 200
            scheduler_threshold = 200
            tol = 0
            history_step = 1000
        
        elif optimizer_type == "LBFGS":
#             for key in train_dataloader.dataset.target.keys:
#                 optimizer_dict[str(key)] = torch.optim.LBFGS(self.models[str(key)].parameters(), lr = lr)
            optimizer_dict[0] = torch.optim.LBFGS(self.models.parameters(), lr = lr)
            threshold = 30
            scheduler_threshold = 30
            tol = 0
            history_step = 10                
    
        scheduler_dict = {}
        scheduler_dict[0] = torch.optim.lr_scheduler.StepLR(optimizer_dict[0], scheduler_threshold, gamma = 0.5)
#         for key in train_dataloader.dataset.target.keys:
#             scheduler_dict[str(key)] = torch.optim.lr_scheduler.StepLR(optimizer_dict[str(key)], scheduler_threshold, gamma = 0.5)

        reg_weights = torch.tensor(list(regularization_dict.values()))
        best_state = copy.deepcopy(self.state_dict())
        lowest_loss = torch.tensor(9999)
        pred_loss = torch.tensor(0)
        trigger = 0
        loss_history = []
        pbar = tqdm(range(n_epochs))
        
        for epoch in pbar:
            pbar.set_description(f"Epoch: {epoch}")
            pbar.set_postfix(pred_loss = pred_loss.item(), lowest_loss = lowest_loss.item(), trigger = trigger)
            train_dataloader.dataset.currentkey = None
            
            for x_data, y_data, structure in train_dataloader: 
                self.collective_zg(optimizer_dict)
                #x_data, y_data = x_data.to(self.device), y_data.to(self.device)
                if optimizer_type == "LBFGS":
                    def closure():
                        self.collective_zg(optimizer_dict)
                        _pred = self.forward(x_data)
                        _pred_loss = loss_function(_pred, y_data, structure, orbs)       
                        _pred_loss = torch.nan_to_num(_pred_loss, nan=lowest_loss.item(), posinf = lowest_loss.item(), neginf = lowest_loss.item())                          
                        _reg_loss = self.get_regression_values(reg_weights) #Only works for 1 layer #Need to change!!
                        _new_loss = _pred_loss + _reg_loss
                        _new_loss.backward()
                        return _new_loss
                    for value in optimizer_dict.values():
                        value.step(closure)
#                     for param in self.parameters():
#                         print (param.grad)
                elif optimizer_type == "Adam":
                    pred = self.forward(x_data)
                    pred_loss = loss_function(pred, y_data, structure, orbs)  
#                     reg_loss = torch.sum(torch.pow(self.nn.weight,2))#Only works for 1 layer
                    new_loss = pred_loss 
                    new_loss.backward()
                    self.collective_step(optimizer_dict)
            with torch.no_grad():
                current_loss = 0 
                for x_data, y_data, structure in train_dataloader:
                    pred = self.forward(x_data)
                    current_loss  += loss_function(pred, y_data, structure, orbs)   #Loss should be normalized already
                pred_loss = current_loss
                reg_loss = self.get_regression_values(reg_weights)#Only works for 1 layer
                new_loss = pred_loss + reg_loss

                if pred_loss >100000 or (pred_loss.isnan().any()) :
                    print ("Optimizer shows weird behaviour, reinitializing at previous best_State")
                    self.load_state_dict(best_state)
                    if optimizer_type == "Adam":
                        optimizer = torch.optim.Adam(self.parameters(), lr = lr, weight_decay = reg.item())
                    elif optimizer_type == "LBFGS":
                        optimizer = torch.optim.LBFGS(self.parameters(), lr = lr)

                if epoch % history_step == 1:
                    loss_history.append(lowest_loss.item())
                
                if lowest_loss - new_loss > tol: #threshold to stop training             
                    best_state = copy.deepcopy(self.state_dict())
                    lowest_loss = new_loss 
                    trigger = 0 
                    
                    
                else:
                    trigger += 1
                    self.collective_step(scheduler_dict)
                    if trigger > threshold:
                        self.load_state_dict(best_state)
                        print ("Implemented early stopping with lowest_loss: {}".format(lowest_loss))
                        return loss_history
        return loss_history
        
    def collective_step(self, dictionary):
        for value in dictionary.values():
            value.step()
            
    def collective_zg(self, dictionary):
        for value in dictionary.values():
            value.zero_grad()
    
    def get_regression_values(self, reg_weights):
        output = []
        for param in self.parameters():
            output.append(torch.sum(torch.pow(param,2)))
        try:
            output = torch.sum(torch.tensor(output) * reg_weights)
        except:
            output = 0
        return output


In [337]:
class BlockModel(torch.nn.Module): #Currently only 1 model per block
    def __init__(self, reconstruction_function, inputSize, outputSize, device, keys, target_key, seed = None, hidden_layers = None):
        super().__init__()
        self.reconstruction_function = reconstruction_function
        self.inputSize = inputSize
        self.outputSize = outputSize
        self.device = device
        self.keys = keys
        self.target_key = target_key
        self.hidden_layers = hidden_layers
        self.initialize_model(seed)
        
        self.to(device)
    
    def initialize_model(self, seed):
        
        if seed is not None:
            torch.manual_seed(seed)
        
        self.models = torch.nn.ModuleDict()
        for index, key in enumerate(self.keys):
            self.models[str(key)] = torch.nn.Linear(self.inputSize[index], self.outputSize[index], bias = False)
        
    def forward(self, feature_tensormap):
        #Block model uses feature keys
        pred_values = {}
        for key in self.keys:
            feature_values = feature_tensormap.block(key).values
            d1, d2, d3 = feature_values.shape
            L = int((d2 -1)/2)
            pred = self.models[str(key)](torch.tensor(feature_values.reshape(d1 * d2, d3)))
            pred = pred.reshape(d1,d2)
            pred_values[L] = pred
        
        pred_block_values = self.reconstruction_function({(self.target_key['l_i'],self.target_key['l_j']) : pred_values})
        

        #pred = torch.hstack(pred_values)
        #pred = self.reconstruction_function(pred_values)
        return pred_block_values 

    
    def fit(self,traindata_loader, loss_function, optimizer_type, lr, reg, n_epochs):
        if optimizer_type == "Adam":
            optimizer = torch.optim.Adam(self.parameters(), lr = lr, weight_decay = reg.item())
            threshold = 200
            scheduler_threshold = 50
            tol = 0
            history_step = 1000
        
        elif optimizer_type == "LBFGS":
            optimizer = torch.optim.LBFGS(self.parameters(), lr = lr)
            threshold = 30
            scheduler_threshold = 10
            tol = 0
            history_step = 10
            
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, scheduler_threshold, gamma = 0.5)
        best_state = copy.deepcopy(self.state_dict())
        lowest_loss = torch.tensor(9999)
        pred_loss = torch.tensor(0)
        trigger = 0
        loss_history = []
        pbar = tqdm(range(n_epochs))
        
        for epoch in pbar:
            pbar.set_description(f"Epoch: {epoch}")
            pbar.set_postfix(pred_loss = pred_loss.item(), lowest_loss = lowest_loss.item(), trigger = trigger)
            
            for x_data, y_data, structure in traindata_loader: 
                optimizer.zero_grad()
                #x_data, y_data = x_data.to(self.device), y_data.to(self.device)
                if optimizer_type == "LBFGS":
                    def closure():
                        optimizer.zero_grad()
                        _pred = self.forward(x_data)                                        
                        _pred_loss = loss_function(_pred, y_data.values)
                        _pred_loss = torch.nan_to_num(_pred_loss, nan=lowest_loss.item(), posinf = lowest_loss.item(), neginf = lowest_loss.item())                 
                        _reg_loss = self.get_regression_values(reg.item()) #Only works for 1 layer
                        _new_loss = _pred_loss + _reg_loss
                        _new_loss.backward()
                        return _new_loss
                    optimizer.step(closure)

                elif optimizer_type == "Adam":
                    pred = self.forward(x_data)
                    pred_loss = loss_function(pred, y_data.values)
                    #reg_loss = self.get_regression_values(reg.item())#Only works for 1 layer
                    new_loss = pred_loss #+ reg_loss
                    new_loss.backward()

                    optimizer.step()
                
            with torch.no_grad():
                current_loss = 0 
                for x_data, y_data, structure in traindata_loader:
                    pred = self.forward(x_data)
                    current_loss  += loss_function(pred, y_data.values) #Loss should be normalized already
                pred_loss = current_loss
                reg_loss = self.get_regression_values(reg.item()) 
                new_loss = pred_loss + reg_loss
                if pred_loss >100000 or (pred_loss.isnan().any()) :
                    print ("Optimizer shows weird behaviour, reinitializing at previous best_State")
                    self.load_state_dict(best_state)
                    if optimizer_type == "Adam":
                        optimizer = torch.optim.Adam(self.parameters(), lr = lr, weight_decay = reg.item())
                    elif optimizer_type == "LBFGS":
                        optimizer = torch.optim.LBFGS(self.parameters(), lr = lr)

                if epoch % history_step == 1:
                    loss_history.append(lowest_loss.item())
                
                if lowest_loss - new_loss > tol: #threshold to stop training
                    best_state = copy.deepcopy(self.state_dict())
                    lowest_loss = new_loss 
                    trigger = 0 
                    
                    
                else:
                    trigger += 1
                    scheduler.step()
                    if trigger > threshold:
                        self.load_state_dict(best_state)
                        print ("Implemented early stopping with lowest_loss: {}".format(lowest_loss))
                        return loss_history
        return loss_history
    
    def get_regression_values(self, reg_weights):
        output = []
        for param in self.parameters():
            output.append(torch.sum(torch.pow(param,2)))
        try:
            output = torch.sum(torch.tensor(output) * reg_weights)
        except:
            output = 0
        return output

## Training

In [510]:
def mse_block_values(pred, true):
    true = true.reshape(true.shape[:-1]) 
    MSE = torch.sum(torch.pow(true - pred,2)) / torch.numel(true)
    return MSE*10**7

def mse_full(pred_blocks, fock,frame, orbs):
    predicted = blocks_to_dense(pred_blocks, frame, orbs)
    #fock = torch.tensor(focks)
    #print (mse_full_blockwise(pred_blocks, blocks, frame, orbs))
    mse_loss = torch.empty(len(frame))
    for i in range(len(frame)):
        mse_loss[i] = ((torch.linalg.norm(fock[i]-predicted[i]))**2)/torch.numel(fock[i])
        #print("from mse", i, fock[i], mse_loss[i])
    return torch.mean(mse_loss)*(Hartree)**2, mse_loss

def mse_full_blockwise(pred_blocks, block_tensormap, frame, orbs):
    indiv_mse = torch.zeros(1)
    for key,block in pred_blocks:
        #MSE = ((torch.linalg.norm(block_tensormap.block(key).values-block.values))**2)
        print (key)
        print (((torch.linalg.norm(block_tensormap.block(key).values- block.values))**2)/ torch.numel(block_tensormap.block(key).values))
        print (torch.sum(torch.pow(block_tensormap.block(key).values - block.values, 2)) / torch.numel(block_tensormap.block(key).values))
        MSE = torch.sum(torch.pow(block_tensormap.block(key).values - block.values, 2)) / torch.numel(block_tensormap.block(key).values)
        indiv_mse += MSE
    
    return (indiv_mse/len(frame))*(Hartree)**2, indiv_mse
    
def mse_eigvals(pred_blocks, fock, frame, orbs):
    fock = torch.tensor(focks)
    predicted = blocks_to_dense(pred_blocks, frame, orbs)
    evanorm = torch.empty(len(frame))
    for i in range(len(frame)):
        evanorm[i] = torch.mean((torch.linalg.eigvalsh(fock[i]) - torch.linalg.eigvalsh(predicted[i]))**2)/len(fock[i])
    return torch.mean(evanorm)*(Hartree)**2

In [477]:
blocks_zero = dense_to_blocks(np.zeros_like(focks), frames, orbs)

In [478]:
blocks_ones = dense_to_blocks(np.ones_like(focks), frames, orbs)

In [511]:
a,  b = mse_full(blocks_ones,torch.zeros_like(torch.tensor(focks)), frames, orbs)
print (a)
print (b)

tensor(740.4595)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])


In [519]:
blocks_ones.keys

Labels([(0, 8, 2, 0, 8, 2, 0), (0, 8, 2, 0, 8, 2, 1),
        (0, 8, 2, 1, 8, 2, 1), (2, 1, 1, 0, 8, 2, 0),
        (2, 1, 1, 0, 8, 2, 1), (0, 1, 1, 0, 1, 1, 0),
        (1, 1, 1, 0, 1, 1, 0)],
       dtype=[('block_type', '<i4'), ('a_i', '<i4'), ('n_i', '<i4'), ('l_i', '<i4'), ('a_j', '<i4'), ('n_j', '<i4'), ('l_j', '<i4')])

In [509]:
blocks_zero.block(6).values.shape

torch.Size([50, 1, 1, 1])

In [401]:
testham = HamModel(test, "cpu")
regularization_dict = {}
# for a in range (16):
#     regularization_dict[a] = torch.tensor(0)
# for value in testham.models.values():
#     for key in value.models:

#         regularization_dict[str(key)] = torch.tensor(0)
# print (len(regularization_dict))

for key in blocks.keys:
    regularization_dict[str(key)] = torch.tensor(0)
#testham.train_individual(dataloader, regularization_dict, "LBFGS", 1000, mse_block_values, 1)
testham.train_collective(dataloader, regularization_dict, "LBFGS", 4, mse_full, 1)


Epoch: 3: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.98s/it, lowest_loss=0.0556, pred_loss=0.0556, trigger=0]


[2.0267622572189157]

In [402]:
testham2 = HamModel(test, "cpu")
regularization_dict = {}
# for a in range (16):
#     regularization_dict[a] = torch.tensor(0)
# for value in testham.models.values():
#     for key in value.models:

#         regularization_dict[str(key)] = torch.tensor(0)
# print (len(regularization_dict))

for key in blocks.keys:
    regularization_dict[str(key)] = torch.tensor(0)
#testham.train_individual(dataloader, regularization_dict, "LBFGS", 1000, mse_block_values, 1)
testham2.train_collective(dataloader, regularization_dict, "LBFGS", 4, mse_full_blockwise, 1)

Epoch: 3: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.20it/s, lowest_loss=0.0416, pred_loss=0.0416, trigger=0]


[1.4445156106576071]

In [373]:
regularization_dict = {}
for key in blocks.keys:
    regularization_dict[str(key)] = torch.tensor(0)
testham = HamModel(test, "cpu")
testham.train_individual(dataloader, regularization_dict, "LBFGS", 50, mse_block_values, 1)


Now training on Block 0 of 7


Epoch: 40:  80%|███████████████████████████████████████████████████████████████████████████████████████▏                     | 40/50 [00:01<00:00, 36.60it/s, lowest_loss=0.000869, pred_loss=0.000869, trigger=30]


Implemented early stopping with lowest_loss: 0.0008685962322613775
Now training on Block 1 of 7


Epoch: 41:  82%|███████████████████████████████████████████████████████████████████████████████████████████                    | 41/50 [00:01<00:00, 26.76it/s, lowest_loss=0.00139, pred_loss=0.00139, trigger=30]


Implemented early stopping with lowest_loss: 0.0013860750660304058
Now training on Block 2 of 7


...

Now training on Block 5 of 7


Epoch: 47:  94%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▏      | 47/50 [00:01<00:00, 28.64it/s, lowest_loss=0.0333, pred_loss=0.0333, trigger=30]


Implemented early stopping with lowest_loss: 0.033257556093282035
Now training on Block 6 of 7


Epoch: 40:  80%|███████████████████████████████████████████████████████████████████████████████████████▏                     | 40/50 [00:01<00:00, 38.05it/s, lowest_loss=0.000327, pred_loss=0.000327, trigger=30]

Implemented early stopping with lowest_loss: 0.0003266998871763507


In [341]:
blocks

TensorMap with 7 blocks
keys: ['block_type' 'a_i' 'n_i' 'l_i' 'a_j' 'n_j' 'l_j']
            0        8     2     0     8     2     0
            0        8     2     0     8     2     1
            0        8     2     1     8     2     1
            2        1     1     0     8     2     0
            2        1     1     0     8     2     1
            0        1     1     0     1     1     0
            1        1     1     0     1     1     0

## Evaluation

In [356]:
#Load test set
test_frames1 = ase.io.read("data/water-hamiltonian/water_coords_1000.xyz","50:80")
# frames2 = ase.io.read("data/ethanol-hamiltonian/ethanol_4500.xyz", ":2")
test_frames = test_frames1 #+ frames2
for f in test_frames:
    f.cell = [100,100,100]
    f.positions += 50

# test_focks1 = np.load("data/water-hamiltonian/water_saph_orthogonal.npy", allow_pickle=True)[50:60]
# focks2 = np.load("data/ethanol-hamiltonian/ethanol_saph_orthogonal.npy", allow_pickle = True)[:len(frames2)]

test_focks1 = np.load("data/water-hamiltonian/water_saph_orthogonal.npy", allow_pickle=True)[50:80]
test_focks = test_focks1
# test_focks = np.load("data/water-hamiltonian/water_fock.npy", allow_pickle=True)[50:80]
# test_overlap = np.load("data/water-hamiltonian/water_overlap.npy", allow_pickle=True)[50:80]

# test_orthogonal = []
# for i in range(len(test_focks)): 
#     test_focks[i] = fix_pyscf_l1(test_focks[i],test_frames[i], orbs)
#     test_overlap[i] = fix_pyscf_l1(test_overlap[i],test_frames[i], orbs)
#     test_orthogonal.append(lowdin_orthogonalize(test_focks[i], test_overlap[i]))
# test_focks = np.asarray(test_orthogonal, dtype=np.float64)
    
test_blocks = dense_to_blocks(test_focks, test_frames, orbs)
test_fock_bc = couple_blocks(test_blocks, cg)

In [357]:
test_rhoi = spex.compute(test_frames)
test_gij = pairs.compute(test_frames)
test_rho1i = acdc_standardize_keys(test_rhoi)
test_rho1i.keys_to_properties(['species_neighbor'])
test_gij =  acdc_standardize_keys(test_gij)
test_rho2i = cg_increment(test_rho1i, test_rho1i, lcut=lmax, other_keys_match=["species_center"], clebsch_gordan=cg)
test_rho1ij = cg_increment(test_rho1i, test_gij, lcut=lmax, other_keys_match=["species_center"], clebsch_gordan=cg)

test_features = hamiltonian_features(test_rho2i, test_rho1ij)

In [358]:
from equistore.io import save
save("test_feature.npz", test_features)

In [359]:
# norm_test_feat = normalize_feats(test_features)
# from equistore.io import save
# save("./norm_testfeat.npz", norm_test_feat)

In [360]:
#test_target_path = "./test_fock.npz"
test_feature_path = "./test_feature.npz"
testing = HamiltonianDataset(test_feature_path, test_blocks, test_frames)

In [361]:
from torch.utils.data import DataLoader, BatchSampler, SubsetRandomSampler


#Sampler = torch.utils.data.SubsetRandomSampler(range(1,len(test)+1), generator=None)
test_Sampler = torch.utils.data.sampler.RandomSampler(testing)
test_BSampler = torch.utils.data.sampler.BatchSampler(test_Sampler, batch_size = 50, drop_last = False)

test_dataloader = DataLoader(testing, sampler = test_BSampler, collate_fn = collate_blocks)

In [393]:
def mse_block_values(pred, true):
    true = true.reshape(true.shape[:-1])
    MSE = torch.sum(torch.pow(true - pred,2)) / torch.numel(true)
    return MSE

def mse_full(pred_blocks, fock,frame, orbs):
    predicted = blocks_to_dense(pred_blocks, frame, orbs)
    #fock = torch.tensor(focks)
    mse_loss = torch.empty(len(frame))
    for i in range(len(frame)):
        mse_loss[i] = ((torch.linalg.norm(fock[i]-predicted[i]))**2)#/torch.numel(fock[i])

        #print("from mse", i, fock[i], mse_loss[i])
    return torch.mean(mse_loss)*(Hartree)**2

def mse_eigvals(pred_blocks, fock, frame, orbs):
    #fock = torch.tensor(focks)
    predicted = blocks_to_dense(pred_blocks, frame, orbs)
    evanorm = torch.empty(len(frame))
    for i in range(len(frame)):
        evanorm[i] = torch.mean((torch.linalg.eigvalsh(fock[i]) - torch.linalg.eigvalsh(predicted[i]))**2)/len(fock[i])
    return torch.mean(evanorm)*(Hartree)**2

In [405]:
dataloader.dataset.currentkey = None
for x_data, y_data, structures in dataloader:
    t_pred = testham(x_data)
    print (mse_full(t_pred, torch.tensor(focks), structures, orbs))

tensor(0.0226, grad_fn=<MulBackward0>)


In [406]:
dataloader.dataset.currentkey = None
for x_data, y_data, structures in dataloader:
    t_pred = testham2(x_data)
    print (mse_full(t_pred, torch.tensor(focks), structures, orbs))

tensor(0.0348, grad_fn=<MulBackward0>)


In [407]:
dataloader.dataset.currentkey = None
for x_data, y_data, structures in dataloader:
    t_pred = testham(x_data)
    print (mse_full_blockwise(t_pred, test.target, structures, orbs))

tensor([0.0165], grad_fn=<MulBackward0>)


In [408]:
dataloader.dataset.currentkey = None
for x_data, y_data, structures in dataloader:
    t_pred = testham2(x_data)
    print (mse_full_blockwise(t_pred, test.target, structures, orbs))

tensor([0.0224], grad_fn=<MulBackward0>)


In [378]:
test_dataloader.dataset.currentkey = None
for x_data, y_data, structures in test_dataloader:
    pred = testham(x_data)
    print (mse_full(pred, torch.tensor(test_focks), structures, orbs))

tensor(57.9591, grad_fn=<MulBackward0>)


In [371]:
testing.currentkey = None
x_data, y_data, structures = testing[[a for a in range(30)]]
pred = testham(x_data)
print (mse_full(pred, torch.tensor(test_focks), test_frames, orbs))

AttributeError: 'Tensor' object has no attribute 'block'

In [353]:
test.currentkey = None
x_data, y_data, structures = test[[a for a in range(1)]]
t_pred = testham(x_data)
print (mse_full(t_pred, torch.tensor(focks), frames, orbs))

tensor(57.3932, grad_fn=<MulBackward0>)


In [60]:
predicted = blocks_to_dense(pred, test_frames, orbs)

In [64]:
t_predicted = blocks_to_dense(t_pred, structures, orbs)

## Tests 

### set up wigner-d rotations matrices 

In [57]:

from rascal.utils.cg_utils import _wigner_d as wigner_d
lmax=3
I_SQRT_2 = 1.0 / np.sqrt(2)
SQRT_2 = np.sqrt(2)
def _r2c(sp):
    """Real to complex SPH. Assumes a block with 2l+1 reals corresponding
    to real SPH with m indices from -l to +l"""
    l = (len(sp) - 1) // 2  # infers l from the vector size
    rc = np.zeros(len(sp), dtype=np.complex128)
    rc[l] = sp[l]
    for m in range(1, l + 1):
        rc[l + m] = (sp[l + m] + 1j * sp[l - m]) * I_SQRT_2 * (-1) ** m
        rc[l - m] = (sp[l + m] - 1j * sp[l - m]) * I_SQRT_2
    return rc
def _c2r(cp):
    """Complex to real SPH. Assumes a block with 2l+1 complex
    corresponding to Y^m_l with m indices from -l to +l"""
    l = (len(cp) - 1) // 2  # infers l from the vector size
    rs = np.zeros(len(cp), dtype=np.float64)
    rs[l] = np.real(cp[l])
    for m in range(1, l + 1):
        rs[l - m] = (-1) ** m * SQRT_2 * np.imag(cp[l + m])
        rs[l + m] = (-1) ** m * SQRT_2 * np.real(cp[l + m])
    return rs
r2c_mats = {}
c2r_mats = {}
for L in range(0, lmax + 1):
    r2c_mats[L] = np.hstack(
        [_r2c(np.eye(2 * L + 1)[i])[:, np.newaxis] for i in range(2 * L + 1)]
    )
    c2r_mats[L] = np.conjugate(r2c_mats[L]).T
_wddict1 = {}
_wddict2 = {}
for L in range(0, lmax + 1):
    wig1 = wigner_d(L, *rotations[1])
    wig2 = wigner_d(L, *rotations[2])
    _wddict1[L] = np.real(c2r_mats[L] @ np.conjugate(wig1) @ r2c_mats[L])
    _wddict2[L] = np.real(c2r_mats[L] @ np.conjugate(wig2) @ r2c_mats[L])

NameError: name 'rotations' is not defined

### block wise rotations

In [ ]:
cg_coefs_list = []
pred_block_list = []
for key in testham.target_keys:
    test.currentkey = key
    x_data, y_data, structures = test[0,1,2]
    cg_coefs, pred_block = testham.models[str(key)].get_cg_coefs(x_data)
    cg_coefs_list.append(cg_coefs)
    pred_block_list.append(pred_block)

In [425]:
torch.linalg.norm( torch.tensor(_wddict1[1]) @ cg_coefs_list[1][1][0] - cg_coefs_list[1][1][1])
torch.linalg.norm( torch.tensor(_wddict2[1]) @ cg_coefs_list[1][1][0] - cg_coefs_list[1][1][2])

tensor(2.2676e-11, grad_fn=<LinalgVectorNormBackward0>)

In [432]:
torch.linalg.norm( torch.tensor(_wddict1[2]) @ cg_coefs_list[2][2][0] - cg_coefs_list[2][2][1])
torch.linalg.norm( torch.tensor(_wddict2[2]) @ cg_coefs_list[2][2][0] - cg_coefs_list[2][2][2])

tensor(5.0514e-11, grad_fn=<LinalgVectorNormBackward0>)

In [434]:
cg_coefs_list

[{0: tensor([[-0.0088],
          [-0.0088],
          [-0.0088]], grad_fn=<ReshapeAliasBackward0>)},
 {1: tensor([[ 0.0000, -0.0057,  0.0000],
          [-0.0026, -0.0049, -0.0012],
          [-0.0009, -0.0029,  0.0048]], grad_fn=<ReshapeAliasBackward0>)},
 {0: tensor([[0.0085],
          [0.0085],
          [0.0085]], grad_fn=<ReshapeAliasBackward0>),
  2: tensor([[ 0.0000,  0.0000,  0.0005,  0.0000, -0.0056],
          [ 0.0050, -0.0006,  0.0013, -0.0022, -0.0003],
          [ 0.0005, -0.0007, -0.0036, -0.0030, -0.0031]],
         grad_fn=<ReshapeAliasBackward0>)},
 {0: tensor([[-0.0022],
          [-0.0022],
          [-0.0022],
          [-0.0022],
          [-0.0022],
          [-0.0022]], grad_fn=<ReshapeAliasBackward0>)},
 {1: tensor([[-0.0031,  0.0021,  0.0000],
          [ 0.0031,  0.0021,  0.0000],
          [ 0.0028,  0.0003,  0.0024],
          [-0.0009,  0.0033, -0.0016],
          [ 0.0034,  0.0007, -0.0014],
          [-0.0027,  0.0014, -0.0022]], grad_fn=<ReshapeAliasB

### Rotate matrix

In [ ]:

predicted_np = [i.detach().numpy() for i in predicted]

pred_bc = couple_blocks(dense_to_blocks(predicted_np, frames, orbs), cg)


### Eigenvalue tests

In [444]:
testham.currentkey = None
x_data, y_data, structures = test[0,1,2]
pred = testham(x_data)
predicted = blocks_to_dense(pred, frames, orbs)


for i in blocks_to_dense(blocks, frames, orbs):
    print (torch.linalg.eigvalsh(i))

np.linalg.eigvalsh(focks)

for i in predicted:
    print (torch.linalg.eigvalsh(i))

tensor([-1.3136, -0.6966, -0.5671, -0.4974,  0.1744,  0.2537])
tensor([-1.3136, -0.6966, -0.5671, -0.4974,  0.1744,  0.2537])
tensor([-1.3136, -0.6966, -0.5671, -0.4974,  0.1744,  0.2537])
tensor([-1.2385e-04, -9.3104e-05, -8.7328e-05, -5.9675e-05, -9.9897e-06,
        -2.1660e-06], grad_fn=<LinalgEighBackward0>)
tensor([-1.2385e-04, -9.3104e-05, -8.7328e-05, -5.9675e-05, -9.9897e-06,
        -2.1660e-06], grad_fn=<LinalgEighBackward0>)
tensor([-1.2385e-04, -9.3104e-05, -8.7328e-05, -5.9675e-05, -9.9897e-06,
        -2.1660e-06], grad_fn=<LinalgEighBackward0>)


tensor([-1.3136, -0.6966, -0.5671, -0.4974,  0.1744,  0.2537])
tensor([-1.3136, -0.6966, -0.5671, -0.4974,  0.1744,  0.2537])
tensor([-1.3136, -0.6966, -0.5671, -0.4974,  0.1744,  0.2537])
tensor([-2.4574, -1.6094, -0.6429, -0.3724, -0.3137,  0.5011],
       grad_fn=<LinalgEighBackward0>)
tensor([-2.4574, -1.6094, -0.6428, -0.3724, -0.3137,  0.5012],
       grad_fn=<LinalgEighBackward0>)
tensor([-2.4574, -1.6094, -0.6429, -0.3724, -0.3137,  0.5011],
       grad_fn=<LinalgEighBackward0>)


### check decoupling of blocks 

In [245]:
couple={}
couple[0] = fock_bc.block(2).values[0].swapaxes(-2,-1)
couple[2] = fock_bc.block(3).values[0].swapaxes(-2,-1)

decoupled = cg.decouple( {(1,1):couple})


In [271]:
np.linalg.norm( torch.tensor(_wddict[1]) @ fock_bc.block(1).values[0] - fock_bc.block(1).values[1])

1.207001048246386e-14

### check feature rotations

In [560]:
np.linalg.norm( _wddict[1] @ features.block(1).values[0] - features.block(1).values[2]) / np.linalg.norm(features.block(1).values[0])

9.91781977311388e-09

In [79]:
np.linalg.norm( _wddict[1] @ norm_feat.block(1).values[0] - norm_feat.block(1).values[2])/np.linalg.norm(norm_feat.block(1).values[0])

NameError: name '_wddict' is not defined

In [664]:
features.block(0).samples

Labels([( 0, 1, 1), ( 0, 2, 2), ( 1, 1, 1), ( 1, 2, 2), ( 2, 1, 1),
        ( 2, 2, 2), ( 3, 1, 1), ( 3, 2, 2), ( 4, 1, 1), ( 4, 2, 2),
        ( 5, 1, 1), ( 5, 2, 2), ( 6, 1, 1), ( 6, 2, 2), ( 7, 1, 1),
        ( 7, 2, 2), ( 8, 1, 1), ( 8, 2, 2), ( 9, 1, 1), ( 9, 2, 2),
        (10, 1, 1), (10, 2, 2), (11, 1, 1), (11, 2, 2), (12, 1, 1),
        (12, 2, 2), (13, 1, 1), (13, 2, 2), (14, 1, 1), (14, 2, 2),
        (15, 1, 1), (15, 2, 2), (16, 1, 1), (16, 2, 2), (17, 1, 1),
        (17, 2, 2), (18, 1, 1), (18, 2, 2), (19, 1, 1), (19, 2, 2),
        (20, 1, 1), (20, 2, 2), (21, 1, 1), (21, 2, 2), (22, 1, 1),
        (22, 2, 2), (23, 1, 1), (23, 2, 2), (24, 1, 1), (24, 2, 2),
        (25, 1, 1), (25, 2, 2), (26, 1, 1), (26, 2, 2), (27, 1, 1),
        (27, 2, 2), (28, 1, 1), (28, 2, 2), (29, 1, 1), (29, 2, 2),
        (30, 1, 1), (30, 2, 2), (31, 1, 1), (31, 2, 2), (32, 1, 1),
        (32, 2, 2), (33, 1, 1), (33, 2, 2), (34, 1, 1), (34, 2, 2),
        (35, 1, 1), (35, 2, 2), (36, 1, 1), (36,

In [673]:
blocks.block(5).samples

Labels([( 1, 1, 1), ( 1, 2, 2), ( 2, 1, 1), ( 2, 2, 2), ( 3, 1, 1),
        ( 3, 2, 2), ( 4, 1, 1), ( 4, 2, 2), ( 5, 1, 1), ( 5, 2, 2),
        ( 6, 1, 1), ( 6, 2, 2), ( 7, 1, 1), ( 7, 2, 2), ( 8, 1, 1),
        ( 8, 2, 2), ( 9, 1, 1), ( 9, 2, 2), (10, 1, 1), (10, 2, 2),
        (11, 1, 1), (11, 2, 2), (12, 1, 1), (12, 2, 2), (13, 1, 1),
        (13, 2, 2), (14, 1, 1), (14, 2, 2), (15, 1, 1), (15, 2, 2),
        (16, 1, 1), (16, 2, 2), (17, 1, 1), (17, 2, 2), (18, 1, 1),
        (18, 2, 2), (19, 1, 1), (19, 2, 2), (20, 1, 1), (20, 2, 2),
        (21, 1, 1), (21, 2, 2), (22, 1, 1), (22, 2, 2), (23, 1, 1),
        (23, 2, 2), (24, 1, 1), (24, 2, 2), (25, 1, 1), (25, 2, 2),
        (26, 1, 1), (26, 2, 2), (27, 1, 1), (27, 2, 2), (28, 1, 1),
        (28, 2, 2), (29, 1, 1), (29, 2, 2), (30, 1, 1), (30, 2, 2),
        (31, 1, 1), (31, 2, 2), (32, 1, 1), (32, 2, 2), (33, 1, 1),
        (33, 2, 2), (34, 1, 1), (34, 2, 2), (35, 1, 1), (35, 2, 2),
        (36, 1, 1), (36, 2, 2), (37, 1, 1), (37,

In [671]:
fock_bc.block(6).samples

Labels([( 1, 1, 1), ( 1, 2, 2), ( 2, 1, 1), ( 2, 2, 2), ( 3, 1, 1),
        ( 3, 2, 2), ( 4, 1, 1), ( 4, 2, 2), ( 5, 1, 1), ( 5, 2, 2),
        ( 6, 1, 1), ( 6, 2, 2), ( 7, 1, 1), ( 7, 2, 2), ( 8, 1, 1),
        ( 8, 2, 2), ( 9, 1, 1), ( 9, 2, 2), (10, 1, 1), (10, 2, 2),
        (11, 1, 1), (11, 2, 2), (12, 1, 1), (12, 2, 2), (13, 1, 1),
        (13, 2, 2), (14, 1, 1), (14, 2, 2), (15, 1, 1), (15, 2, 2),
        (16, 1, 1), (16, 2, 2), (17, 1, 1), (17, 2, 2), (18, 1, 1),
        (18, 2, 2), (19, 1, 1), (19, 2, 2), (20, 1, 1), (20, 2, 2),
        (21, 1, 1), (21, 2, 2), (22, 1, 1), (22, 2, 2), (23, 1, 1),
        (23, 2, 2), (24, 1, 1), (24, 2, 2), (25, 1, 1), (25, 2, 2),
        (26, 1, 1), (26, 2, 2), (27, 1, 1), (27, 2, 2), (28, 1, 1),
        (28, 2, 2), (29, 1, 1), (29, 2, 2), (30, 1, 1), (30, 2, 2),
        (31, 1, 1), (31, 2, 2), (32, 1, 1), (32, 2, 2), (33, 1, 1),
        (33, 2, 2), (34, 1, 1), (34, 2, 2), (35, 1, 1), (35, 2, 2),
        (36, 1, 1), (36, 2, 2), (37, 1, 1), (37,

### mean Wei Bin's tests 

#### meanless hamiltonian 

In [499]:
tar = focks - np.mean(focks)
tar_bc = couple_blocks(dense_to_blocks(tar, frames, orbs), cg)

In [500]:
tar_bc.block(7).values

tensor([[[0.0046]],

        [[0.0046]],

        [[0.0046]]])

In [501]:
print(torch.linalg.norm( torch.tensor(_wddict1[2]) @ tar_bc.block(3).values[0] - tar_bc.block(3).values[1]))
print(torch.linalg.norm( torch.tensor(_wddict2[2]) @ tar_bc.block(3).values[0] - tar_bc.block(3).values[2]))

tensor(0.4896)
tensor(0.2537)


this **DOESNT** work 

#### meanless blocks

In [488]:
ppblock = blocks.block(2)
block_mean = torch.mean(ppblock.values, axis = 0)
ppblock_m = (ppblock.values - block_mean).squeeze()

In [491]:
pp_m_bc = cg.couple( ppblock_m)

In [495]:
pp_m_bc = pp_m_bc[(1,1)]

In [496]:
print(torch.linalg.norm( torch.tensor(_wddict1[2]) @ pp_m_bc[2][0] - pp_m_bc[2][1]))
print(torch.linalg.norm( torch.tensor(_wddict2[2]) @ pp_m_bc[2][0] - pp_m_bc[2][2]))

tensor(0.2264)
tensor(0.0990)


#### stupid test 

In [502]:
tar = []
for i,f in enumerate(focks): 
    tar.append(f-f.mean())

tar_bc = couple_blocks(dense_to_blocks(tar, frames, orbs), cg)

In [503]:
print(torch.linalg.norm( torch.tensor(_wddict1[2]) @ tar_bc.block(3).values[0] - tar_bc.block(3).values[1]))
print(torch.linalg.norm( torch.tensor(_wddict2[2]) @ tar_bc.block(3).values[0] - tar_bc.block(3).values[2]))

tensor(0.4536)
tensor(0.2722)


## Jigyasa needs to modify samples for features to start from 1

## Old and existing

In [ ]:
def linear_init_weights(in_size, out_size):
    m=torch.nn.Linear(in_size, out_size)
    #m.weight.data.fill_(0)
    m.bias.data.fill_(0)
    return m 

class BlockModel(torch.nn.Module):
    def __init__(self, frames, block, feat, optim, layer_size=None):
        super().__init__()
        self.frames = frames
        self.nn = None
        self.layer_size = layer_size
        self.feature = feat
        self.values = block
        
        
    def initialize_model(self, seed):
        
        if seed is not None:
            torch.manual_seed(seed)
        inputSize = self.feature.values.shape[-1] #feature dimension 
        outputSize = self.block.values.shape[0] #block sample size 
        
        if self.layer_size is None: 
            self.nn = nn.Linear(inputSize, outputSize, bias = False)
            
        else:
#             self.layer1 = nn.Linear(10,10)
#             self.act1 = nn.TanH()
#             self.layer2 = nn.Linear(10,100)
#             self.act2 = nn.Sigmoid()
            self.nn = torch.nn.Sequential(
                linear_init_weights(inputSize, self.layer_size),
                torch.nn.Tanh(),
                linear_init_weights(self.layer_size, self.layer_size),
                torch.nn.Tanh(),
                linear_init_weights(self.layer_size, OutputSize),
            )
           
        
    def forward(self):
        if self.nn is None:
            raise Exception("call initialize_model first")
            
        pred = self.nn(self.feature.values)
#         pred = self.layer1(pred)
#         pred = self.act1(pred)
#         pred = self.layer2(pred)
#         pred = self.act2(pred)
#         pred = self.layer1(pred)
        return pred 

    def fit(loss_function, optimizer_type, lr, reg, nepochs = 5000):
        if optimizer_type == "Adam":
            optimizer = torch.optim.Adam(self.parameters(), lr = lr, weight_decay = self.reg.item())
            threshold = 20000
            scheduler_threshold = 20000
            tol = 1e-3
            
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, scheduler_threshold, gamma = 0.5)
        best_state = copy.deepcopy(self.nn.state_dict())
        
        for epoch in tqdm(nepochs):
            optimizer.zero_grad()
            pred = self.nn(self.feature.values)
            loss = loss_function(self.values, pred, self.frames, )
            loss.backward()
            optimizer.step()
            scheduler.step()

In [ ]:
class LinearModel(torch.nn.Module):
    def __init__(self, coupled_blocks, features, weights=None, intercepts=None):
        super().__init__()
        self.coupled_blocks = coupled_blocks
        self.features = features
        self.weights = {}
        if weights==None:
            for idx_fock, block_fock in self.coupled_blocks:
                block_type, ai, ni, li, aj, nj, lj, L = idx_fock
                parity= (-1)**(li+lj+L)
                size = self.features.block(block_type=block_type, spherical_harmonics_l=L,inversion_sigma=parity, 
                                       species_center=ai, species_neighbor=aj).values.shape[2]
                #self.weights[idx_fock] = torch.nn.Parameter(torch.zeros(size, dtype=torch.float64))
                self.weights[idx_fock] = torch.nn.Parameter(torch.randn(size, dtype=torch.float64))
            
        else: 
            self.weights = weights
        
        self.intercepts = {}
        if intercepts is None:
            for idx_fock, block_fock in self.coupled_blocks:
                block_type, ai, ni, li, aj, nj, lj, L = idx_fock
                parity= (-1)**(li+lj+L)
                if L==0 and parity==1 and block_type==0:
                    self.intercepts[idx_fock] = torch.nn.Parameter(torch.randn(1, dtype=torch.float64))
                else:
                    self.intercepts[idx_fock] = 0
        else:
            self.intercepts = intercepts
         
    def forward(self, features):
        k = []
        pred_blocks = []
        for (idx, wts) in self.weights.items():
            #print(wts)
            block_type, ai, ni, li, aj, nj, lj, L = idx
            k.append(list(idx))
            parity= (-1)**(li+lj+L)
            X = features.block(block_type=block_type, spherical_harmonics_l=L,inversion_sigma=parity, 
                                   species_center=ai, species_neighbor=aj)
            X_new = torch.from_numpy(X.values.reshape(-1, X.values.shape[2]))
            #print(idx, wts.shape, X.values.shape, X_new.shape)
            Y = X_new @ wts + self.intercepts[idx]
            
            newblock = TensorBlock(
                        values=Y.reshape((-1, 2 * L + 1, 1)),
                        samples=X.samples,
                        components=[Labels(
                            ["mu"], np.asarray(range(-L, L + 1), dtype=np.int32).reshape(-1, 1)
                        )],
                        properties= Labels(["values"], np.asarray([[0]], dtype=np.int32))
                    )
            pred_blocks.append(newblock) 
        
        keys = Labels(('block_type', 'a_i', 'n_i', 'l_i', 'a_j', 'n_j', 'l_j', 'L'), np.asarray(k, dtype=np.int32))
        pred_fock = TensorMap(keys, pred_blocks)
        return(pred_fock)
        ### add direct eigenvalue prediction here as well
    
    def parameters(self):
        for idx, wts in self.weights.items():
            yield wts
        for idx, wts in self.intercepts.items():
            if type(wts) is not int:
                yield wts